# 🚀 CDF Toolkit Bootstrap - Interactive Deployment

**Deploys the breakthrough Functions-based toolkit solution**

## What This Deploys

1. **Cognite Function**: `test-toolkit-function`
   - Installs cognite-toolkit in Functions environment
   - Runs real `cdf build`, `cdf deploy` commands
   
2. **Streamlit App**: `test-toolkit-api`
   - UI for calling the function
   - Displays breakthrough results
   
3. **Dataset**: `streamlit-test-toolkit-dataset`
   - Required dataset for Streamlit app

## How to Use

1. Run cells in order (Cell → Run All, or run one-by-one)
2. Watch for ✅ success messages
3. If errors occur, read the message and fix issues
4. After completion, go to CDF UI → Apps → Streamlit

**Source**: https://github.com/bgfast/cognite-quickstart

---

## Cell 1: Setup & Configuration

In [ ]:
# Configuration - GitHub source URLs
GITHUB_RAW_BASE = "https://raw.githubusercontent.com/bgfast/cognite-quickstart/oct_mods"
FUNCTION_HANDLER_URL = f"{GITHUB_RAW_BASE}/modules/test-toolkit-api/functions/test-toolkit-function/handler.py"
FUNCTION_REQS_URL = f"{GITHUB_RAW_BASE}/modules/test-toolkit-api/functions/test-toolkit-function/requirements.txt"
STREAMLIT_MAIN_URL = f"{GITHUB_RAW_BASE}/modules/test-toolkit-api/streamlit/test-toolkit-api/main.py"
STREAMLIT_REQS_URL = f"{GITHUB_RAW_BASE}/modules/test-toolkit-api/streamlit/test-toolkit-api/requirements.txt"

# Resource names (customize if needed)
DATASET_EXTERNAL_ID = "streamlit-test-toolkit-dataset"
FUNCTION_EXTERNAL_ID = "test-toolkit-function"
STREAMLIT_EXTERNAL_ID = "test-toolkit-api"

print("✅ Configuration loaded")
print(f"   Dataset: {DATASET_EXTERNAL_ID}")
print(f"   Function: {FUNCTION_EXTERNAL_ID}")
print(f"   Streamlit: {STREAMLIT_EXTERNAL_ID}")

## Cell 2: Initialize & Verify

In [ ]:
from cognite.client import CogniteClient
import requests
from datetime import datetime

# Initialize CogniteClient
client = CogniteClient()
print("✅ Connected to CDF")
print(f"   Project: {client.config.project}")
print(f"   Cluster: {client.config.base_url}")
print()

# Test GitHub connectivity
try:
    response = requests.get(GITHUB_RAW_BASE, timeout=5)
    print("✅ GitHub connectivity verified")
    print(f"   Source: {GITHUB_RAW_BASE}")
except Exception as e:
    print(f"❌ Cannot reach GitHub: {e}")
    print("💡 Check internet access or try again later")
    raise

## Cell 3: Create Dataset

In [ ]:
print("📦 Step 1/3: Creating dataset...")
print()

# Check if exists
existing = client.data_sets.retrieve(external_id=DATASET_EXTERNAL_ID)

if existing:
    print(f"ℹ️  Dataset already exists")
    print(f"   External ID: {existing.external_id}")
    print(f"   Name: {existing.name}")
    print(f"   Created: {existing.created_time}")
else:
    # Create new dataset
    from cognite.client.data_classes import DataSet
    dataset = client.data_sets.create(
        DataSet(
            external_id=DATASET_EXTERNAL_ID,
            name="Streamlit Test Toolkit Dataset",
            description="Dataset for Streamlit Toolkit API testing - deployed via bootstrap",
            metadata={
                "deployed_at": datetime.now().isoformat(),
                "bootstrap_version": "2.0",
                "source": "github.com/bgfast/cognite-quickstart"
            }
        )
    )
    print(f"✅ Dataset created")
    print(f"   External ID: {dataset.external_id}")
    print(f"   Name: {dataset.name}")
    print(f"   ID: {dataset.id}")

print()
print("✅ Dataset ready")

## Cell 4: Deploy Cognite Function

In [ ]:
print("🔧 Step 2/3: Deploying Cognite Function...")
print()

# Fetch function code from GitHub
print("📥 Fetching function code from GitHub...")
handler_code = requests.get(FUNCTION_HANDLER_URL).text
requirements_txt = requests.get(FUNCTION_REQS_URL).text

print(f"   Handler size: {len(handler_code):,} bytes")
print(f"   Requirements: {len(requirements_txt.splitlines())} packages")
print()

# Deploy function
try:
    # Check if exists and delete
    try:
        existing_func = client.functions.retrieve(external_id=FUNCTION_EXTERNAL_ID)
        client.functions.delete(external_id=FUNCTION_EXTERNAL_ID)
        print(f"🗑️  Removed existing function (id={existing_func.id})")
        print()
    except:
        pass
    
    # Create function package
    print("📦 Creating function package...")
    import tempfile
    import os
    import zipfile
    
    with tempfile.TemporaryDirectory() as tmpdir:
        # Write handler.py
        handler_path = os.path.join(tmpdir, "handler.py")
        with open(handler_path, 'w') as f:
            f.write(handler_code)
        
        # Write requirements.txt
        reqs_path = os.path.join(tmpdir, "requirements.txt")
        with open(reqs_path, 'w') as f:
            f.write(requirements_txt)
        
        # Create zip file
        zip_path = os.path.join(tmpdir, "function.zip")
        with zipfile.ZipFile(zip_path, 'w') as zf:
            zf.write(handler_path, "handler.py")
            zf.write(reqs_path, "requirements.txt")
        
        print(f"   ✅ Package created: {os.path.getsize(zip_path):,} bytes")
        
        # Upload to Files API
        print("📤 Uploading to CDF Files...")
        with open(zip_path, 'rb') as f:
            file_metadata = client.files.upload(
                path=zip_path,
                name=f"function-{FUNCTION_EXTERNAL_ID}.zip",
                external_id=f"function-{FUNCTION_EXTERNAL_ID}-{datetime.now().strftime('%Y%m%d%H%M%S')}"
            )
        
        print(f"   ✅ Uploaded: file_id={file_metadata.id}")
    
    print()
    print("🚀 Creating function in CDF...")
    
    # Create function
    function = client.functions.create(
        name="Test Toolkit Function",
        external_id=FUNCTION_EXTERNAL_ID,
        file_id=file_metadata.id,
        description="Runs cognite-toolkit commands in Functions environment - v2.0",
        owner="deployed-via-bootstrap",
        cpu=0.5,
        memory=1.5,
        runtime="py311",
        metadata={
            "version": "2.0",
            "deployed_at": datetime.now().isoformat(),
            "source": "github-bootstrap"
        }
    )
    
    print(f"✅ Function deployed successfully!")
    print(f"   External ID: {function.external_id}")
    print(f"   Function ID: {function.id}")
    print(f"   Status: {function.status}")
    print(f"   Runtime: {function.runtime}")
    
except Exception as e:
    print(f"❌ Function deployment failed: {e}")
    print("💡 Check Functions quota and permissions")
    raise

print()
print("✅ Function deployment complete")

## Cell 5: Deploy Streamlit App

In [ ]:
print("🎨 Step 3/3: Deploying Streamlit App...")
print()

# Fetch Streamlit code from GitHub
print("📥 Fetching Streamlit code from GitHub...")
main_py = requests.get(STREAMLIT_MAIN_URL).text
streamlit_reqs = requests.get(STREAMLIT_REQS_URL).text

print(f"   main.py size: {len(main_py):,} bytes")
print(f"   requirements: {len(streamlit_reqs.splitlines())} packages")
print()

try:
    # Note: Streamlit API varies by SDK version
    # We'll try multiple approaches
    
    # Clean up existing app
    try:
        # Try different API methods
        try:
            existing_app = client.streamlit.retrieve(external_id=STREAMLIT_EXTERNAL_ID)
            client.streamlit.delete(external_id=STREAMLIT_EXTERNAL_ID)
        except:
            existing_app = client.apps.retrieve(external_id=STREAMLIT_EXTERNAL_ID)
            client.apps.delete(external_id=STREAMLIT_EXTERNAL_ID)
        print(f"🗑️  Removed existing app")
        print()
    except:
        pass
    
    print("🎨 Creating Streamlit app...")
    
    # Get dataset ID
    dataset = client.data_sets.retrieve(external_id=DATASET_EXTERNAL_ID)
    
    # Try new API first
    try:
        streamlit = client.streamlit.upsert(
            external_id=STREAMLIT_EXTERNAL_ID,
            name="Test Toolkit API",
            description="v2.0 - BREAKTHROUGH: Toolkit via Functions",
            data_set_id=dataset.id,
            source_files={
                "main.py": main_py,
                "requirements.txt": streamlit_reqs
            }
        )
    except AttributeError:
        # Fall back to older API
        streamlit = client.apps.create(
            external_id=STREAMLIT_EXTERNAL_ID,
            name="Test Toolkit API",
            description="v2.0 - BREAKTHROUGH: Toolkit via Functions",
            data_set_external_id=DATASET_EXTERNAL_ID,
            files={
                "main.py": main_py,
                "requirements.txt": streamlit_reqs
            }
        )
    
    print(f"✅ Streamlit app deployed successfully!")
    print(f"   External ID: {STREAMLIT_EXTERNAL_ID}")
    print(f"   Name: Test Toolkit API")
    
except Exception as e:
    print(f"❌ Streamlit deployment failed: {e}")
    print("💡 Note: You may need to deploy Streamlit manually via CDF UI")
    print("   The source code is available at the GitHub URLs above")
    # Don't raise - this is non-critical

print()
print("✅ Streamlit deployment complete")

## Cell 6: Verify & Test

In [ ]:
print("="*60)
print("🧪 Verification & Testing")
print("="*60)
print()

# Test function call
print("📞 Calling function to verify it works...")
try:
    call_result = client.functions.call(
        external_id=FUNCTION_EXTERNAL_ID,
        data={"test": "bootstrap_verification"}
    )
    print(f"✅ Function called successfully!")
    print(f"   Call ID: {call_result.id}")
    print(f"   Status: {call_result.status}")
    print()
    print("💡 Function is executing... check CDF UI → Functions for logs")
    print("   The function will:")
    print("   - Install cognite-toolkit")
    print("   - Fix PATH to include cdf commands")
    print("   - Test cdf build, deploy --dry-run, deploy")
    print("   - Return test results (~30 seconds)")
    
except Exception as e:
    print(f"⚠️  Function call failed: {e}")
    print("   Function is deployed but may need time to warm up")
    print("   Try calling again in 1-2 minutes")

print()
print("="*60)
print("🎉 DEPLOYMENT COMPLETE!")
print("="*60)
print()
print("📋 Deployed Resources:")
print(f"   • Dataset: {DATASET_EXTERNAL_ID}")
print(f"   • Function: {FUNCTION_EXTERNAL_ID}")
print(f"   • Streamlit: {STREAMLIT_EXTERNAL_ID}")
print()
print("🚀 Next Steps:")
print("   1. Go to CDF UI → Apps → Streamlit")
print("   2. Find 'Test Toolkit API' app")
print("   3. Open the app")
print("   4. Click 'Call Function to Test Toolkit' button")
print("   5. Wait ~30 seconds for function to complete")
print("   6. See breakthrough confirmation with metrics! 🎊")
print()
print("📚 Learn More:")
print("   • GitHub: https://github.com/bgfast/cognite-quickstart")
print("   • Bootstrap README: ../bootstrap/README.md")
print(f"   • Function Logs: CDF UI → Integrate → Functions → {FUNCTION_EXTERNAL_ID}")
print("   • Documentation: ../cursor-prompts.md")